In [219]:
import torch
from torch import nn,optim
from torchvision import datasets,transforms
from torch.utils.data import dataloader
from tqdm import tqdm


In [220]:
tf = transforms.Compose(
    [transforms.Grayscale(num_output_channels=1), transforms.ToTensor()]
)
# tf = transforms.ToTensor()


In [221]:
training_set = datasets.ImageFolder(root="data/train", transform=tf)
testing_set = datasets.ImageFolder(root="data/test", transform=tf)


In [222]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
BATCH_SIZE = 16


In [223]:
class PyTeen(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers = nn.Sequential(
            nn.Conv2d(1, 6, 3, padding=2),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Conv2d(6, 16, 3),
            nn.ReLU(),
            nn.MaxPool2d(2, stride=2),
            nn.Flatten(),
            nn.Linear(2480, 120),
            nn.ReLU(),
            nn.Linear(120, 84),
            nn.ReLU(),
            nn.Linear(84, 10),
        )
        self.loss = nn.CrossEntropyLoss()
        self.optimizer = optim.Adam(self.parameters())

    # self.to(torch.device(DEVICE)) #gpu

    def forward(self, input):
        return self.layers(input)

    def predict(self, input):
        with torch.no_grad():
            pred = self.forward(input)
            return torch.argmax(pred, axis=-1)

    def train(self, input, label):
        self.optimizer.zero_grad()
        pred = self.forward(input)
        loss = self.loss(pred, label)
        loss.backward()
        self.optimizer.step()
        return loss


In [224]:
training_loader = dataloader.DataLoader(
    training_set, batch_size=BATCH_SIZE, shuffle=True
)
testing_loader = dataloader.DataLoader(
    testing_set, batch_size=BATCH_SIZE, shuffle=False
)


In [225]:
network = PyTeen()
network.to(torch.device(DEVICE))


PyTeen(
  (layers): Sequential(
    (0): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (3): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (6): Flatten(start_dim=1, end_dim=-1)
    (7): Linear(in_features=2480, out_features=120, bias=True)
    (8): ReLU()
    (9): Linear(in_features=120, out_features=84, bias=True)
    (10): ReLU()
    (11): Linear(in_features=84, out_features=10, bias=True)
  )
  (loss): CrossEntropyLoss()
)

In [226]:
# training loop
# we need to make the tensors all .to(torch.device('cuda'))
EPOCHS = 20

for i in range(EPOCHS):
  total_loss = 0
  for input,label in tqdm(training_loader):
      input = input.to(torch.device(DEVICE))
      label = label.to(torch.device(DEVICE))

      loss = network.train(input,label)
      total_loss += loss
  print("EPOCH:",i+1,": ",total_loss)


100%|██████████| 3930/3930 [01:27<00:00, 45.03it/s]


EPOCH: 1 :  tensor(3033.3247, grad_fn=<AddBackward0>)


100%|██████████| 3930/3930 [01:45<00:00, 37.28it/s]


EPOCH: 2 :  tensor(1838.0393, grad_fn=<AddBackward0>)


100%|██████████| 3930/3930 [01:59<00:00, 32.96it/s]


...

EPOCH: 18 :  tensor(547.8857, grad_fn=<AddBackward0>)


100%|██████████| 3930/3930 [01:04<00:00, 61.27it/s]


EPOCH: 19 :  tensor(518.9496, grad_fn=<AddBackward0>)


100%|██████████| 3930/3930 [01:06<00:00, 58.76it/s]

EPOCH: 20 :  tensor(494.9538, grad_fn=<AddBackward0>)


In [227]:
# evaluation loop
# we need to make the tensors all .to(torch.device('cuda'))
num_corrects = 0
for input,label in tqdm(testing_loader):
  input = input.to(torch.device(DEVICE))
  label = label.to(torch.device(DEVICE))
  pred = network.predict(input)
  for i in range(len(pred)):
    if(pred[i] == label[i]):
      num_corrects += 1
print(f"{num_corrects*100/(len(testing_loader)*BATCH_SIZE)}%")


100%|██████████| 378/378 [00:04<00:00, 91.88it/s] 

87.89682539682539%


In [228]:
torch.save(network.state_dict(), "./py_teen.pth")
